In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = pd.read_csv('f1_2014_to_2022_HAM_all_data.csv') # it's for 2018 to 2022


df.dropna(subset=['IsPersonalBest'], inplace=True) # To Handle Nan values

df['Rainfall'] = df['Rainfall'].astype(int) # Encode Rainfall To integer
df['FreshTyre'] = df['FreshTyre'].astype(int) # Encode FreshTyre To integer
df['FastF1Generated'] = df['FastF1Generated'].astype(int)
df['IsPersonalBest'] = df['IsPersonalBest'].astype(int)
df['IsAccurate'] = df['IsAccurate'].astype(int)


df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5630 entries, 0 to 5632
Data columns (total 40 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Time                5630 non-null   object 
 1   Driver              5630 non-null   object 
 2   DriverNumber        5630 non-null   int64  
 3   LapTime             5499 non-null   object 
 4   LapNumber           5630 non-null   float64
 5   Stint               5606 non-null   float64
 6   PitOutTime          166 non-null    object 
 7   PitInTime           167 non-null    object 
 8   Sector1Time         5522 non-null   object 
 9   Sector2Time         5628 non-null   object 
 10  Sector3Time         5624 non-null   object 
 11  Sector1SessionTime  5502 non-null   object 
 12  Sector2SessionTime  5628 non-null   object 
 13  Sector3SessionTime  5624 non-null   object 
 14  SpeedI1             4940 non-null   float64
 15  SpeedI2             5622 non-null   float64
 16  SpeedF

In [2]:
df.groupby(['TrackTemp'])['LapTime'].agg(["count"])

,count
TrackTemp,
13.8,3
13.9,12
14.0,3
14.1,3
14.2,2
...,...
56.8,1
56.9,2
57.0,2


In [3]:
# df['LapTime'] = pd.to_timedelta(df['LapTime'])
""" 
df['LapTime'] = pd.to_timedelta(df['LapTime'])
df['LapTime'] = df['LapTime'].dt.total_seconds()
# Change LapTime to numerical value

corr_matrix = df.corr(numeric_only=True)

# Heatmap to visualize correlations
plt.figure(figsize=(20, 12))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()
"""

"""
# Scatter plot for Track Temperature vs. Lap Time
plt.figure(figsize=(12, 6))
sns.scatterplot(x='TrackTemp', y='LapTime', data=df)
plt.title('Track Temperature vs. Lap Time')
plt.show() """


"\n# Scatter plot for Track Temperature vs. Lap Time\nplt.figure(figsize=(12, 6))\nsns.scatterplot(x='TrackTemp', y='LapTime', data=df)\nplt.title('Track Temperature vs. Lap Time')\nplt.show() "

In [4]:
# sns.histplot(x='LapTime', data=df)

In [5]:
import pandas as pd

# Use pd.get_dummies to convert 'Circuit' column to one-hot encoded variables
one_hot_encoded_data = pd.get_dummies(df['Circuit'], prefix='Circuit')

# Join the one-hot encoded columns back with the original DataFrame
df= df.join(one_hot_encoded_data)

# Now df_one_hot contains the original data along with the one-hot encoded circuit columns

df = df.drop(columns=['Team','Driver','LapStartDate']) # Remove non-related features


df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5630 entries, 0 to 5632
Data columns (total 70 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Time                                 5630 non-null   object 
 1   DriverNumber                         5630 non-null   int64  
 2   LapTime                              5499 non-null   object 
 3   LapNumber                            5630 non-null   float64
 4   Stint                                5606 non-null   float64
 5   PitOutTime                           166 non-null    object 
 6   PitInTime                            167 non-null    object 
 7   Sector1Time                          5522 non-null   object 
 8   Sector2Time                          5628 non-null   object 
 9   Sector3Time                          5624 non-null   object 
 10  Sector1SessionTime                   5502 non-null   object 
 11  Sector2SessionTime            

In [ ]:

plt.figure(figsize=(20, 12))

sns.scatterplot(x='TrackTemp', y='LapTime', data=df, hue='Circuit')


<Axes: xlabel='TrackTemp', ylabel='LapTime'>

In [ ]:
# Handling Outliers

# Calculate Q1, Q3, and IQR

Q1 = df['LapTime'].quantile(0.25)
Q3 = df['LapTime'].quantile(0.75)
IQR = Q3 - Q1

# Determine bounds for outliers
lower_bound = Q1 - 1.5 * IQR  
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = df[(df['LapTime'] < lower_bound) | (df['LapTime'] > upper_bound)]

#  Remove outliers from the DataFrame

df = df[(df['LapTime'] >= lower_bound) & (df['LapTime'] <= upper_bound)]

plt.figure(figsize=(20, 12))
sns.scatterplot(x='TrackTemp', y='LapTime', data=df,hue='Circuit')


plt.title('Track Temperature vs. Lap Time (Outliers Removed)')
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

df['LapTime'] = pd.to_timedelta(df['LapTime'])

df['LapTime'] = df['LapTime'].dt.total_seconds()

# Assuming LapTime is your target variable and is already in a numeric format
X = df.drop('LapTime', axis=1)
y = df['LapTime']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
predictions = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"RMSE: {rmse}")


In [ ]:
# sns.regplot(x='TrackTemp', y='LapTime', data=df)

In [ ]:
# Handling missing values
# You might decide to fill missing values with a median or mean, or drop them
# df.fillna(df.median(), inplace=True ) # For numerical columns


In [ ]:
""" from sklearn.preprocessing import StandardScaler

# Fit the scaler on the selected columns
scaler = StandardScaler().fit(df[['LapTime', 'TrackTemp']])

# Transform the selected columns and store in a new DataFrame
scaled_columns = scaler.transform(df[['LapTime', 'TrackTemp']])

# Create a DataFrame from the scaled columns with the same index as df
scaled_df = pd.DataFrame(scaled_columns, index=df.index, columns=['LapTime_scaled', 'TrackTemp_scaled'])

# Now you can merge the scaled columns back into the original DataFrame
df = df.drop(['LapTime', 'TrackTemp'], axis=1)
df = pd.concat([df, scaled_df], axis=1)
"""

In [ ]:
""" from sklearn.preprocessing import RobustScaler
import pandas as pd

scaler = RobustScaler()

# Fit and transform the columns
scaled_data = scaler.fit_transform(df[['LapTime', 'TrackTemp']])

# Create a DataFrame from the scaled data with the same index as 'df' and new column names
scaled_df = pd.DataFrame(scaled_data, index=df.index, columns=['LapTime_scaled1', 'TrackTemp_scaled2'])

# Drop the original 'LapTime' and 'TrackTemp' columns from 'df' if you no longer need them

df = df.drop(['LapTime', 'TrackTemp'], axis=1)


df = df.drop(['LapTime_scaled', 'TrackTemp_scaled'], axis=1)

# merge the scaled columns back into the original DataFrame
df = pd.concat([df, scaled_df], axis=1)
"""